In [ ]:
!pip install lancedb

In [1]:
import lancedb
import pandas as pd
import pyarrow as pa

uri = "data/sample-lancedb"

db = lancedb.connect(uri)

c:\Users\chyav\Documents\Sakaar\lumona_car\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.chdir('..')

from app.core.parsing.parser import parser

documents = parser.parse_pdf_document(os.path.abspath("C:/Users/chyav/Documents/Sakaar/lumona_car/notebooks/tiago_manual.pdf"))



Processing C:\Users\chyav\Documents\Sakaar\lumona_car\notebooks\tiago_manual.pdf...
[                                        ] (0/15[                                        ] (  1/15[                                        ] (  2/15[                                        ] (  3/153[=                                       ] (  4/153[=                                       ] (  5/153[=                                       ] (  6/1[=                                       ] (  7/15[==                                      ] (  8/15[==                                      ] (  9/15[==                                      ] ( 10/15[==                                      ] ( 11/153[===                                     ] ( 12/153[===                                     ] ( 13/153[===                                     ] ( 14/1[===                                     ] ( 15/15[====                                    ] ( 16/15[====                                    ] ( 17/15[====         

In [6]:
# Check if table exists, if not create it
df = pd.DataFrame(documents)
documents_in_dict = [item.model_dump() for item in documents]
if "cars_docs_data" not in db.table_names():
    # Create table
    db.create_table("cars_docs_data", data=documents_in_dict)
else:
    # Add data to existing table
    table = db.open_table("cars_docs_data")
    table.add(df)


In [25]:
rag_table = db.open_table("cars_docs_data")
fetched_docs = rag_table.to_pandas()

fetched_docs.head()



,id,text,metadata,parent_document_id,type,embedding,keywords,ref_in_parent,tag,previous_document_id,next_document_id
0,7949783dcb314b838fcb409011d7165e,"![](data:image/png;base64,iVBORw0KGgoAAAANSUhE...","{'headings': [], 'images': False, 'tables': Fa...",42f77327d9f14b50848025b4bc8bd4df,PAGE,"[0.024130431935191154, 0.005421559326350689, -...",None,1,OTHER,None,None
1,5ea1b169b5034e3c97de706e4d1e5785,"\n\n![](data:image/png;base64,iVBORw0KGgoAAAAN...","{'headings': [], 'images': False, 'tables': Fa...",42f77327d9f14b50848025b4bc8bd4df,PAGE,"[0.020235136151313782, 0.008043157868087292, -...",None,2,OTHER,7949783dcb314b838fcb409011d7165e,None
2,76e27080698e46d0a6ee83b18bb2254e,\n\nIn our constant endeavour to provide assis...,"{'headings': [], 'images': False, 'tables': Fa...",42f77327d9f14b50848025b4bc8bd4df,PAGE,"[0.0057883476838469505, -0.046952418982982635,...",None,3,OTHER,5ea1b169b5034e3c97de706e4d1e5785,None
3,6a84abdc3e3745519a5232d6d24d4886,"\n\n### Dear Customer,\n\nWelcome to the TATA ...","{'headings': [], 'images': False, 'tables': Fa...",42f77327d9f14b50848025b4bc8bd4df,PAGE,"[0.015528478659689426, -0.07098111510276794, 0...",None,4,OTHER,76e27080698e46d0a6ee83b18bb2254e,None
4,ad92cc51e8614359ae23c3b9c0caccbe,"\n\n- Before driving, read this Owner’s manual...","{'headings': [], 'images': False, 'tables': Fa...",42f77327d9f14b50848025b4bc8bd4df,PAGE,"[0.025854503735899925, -0.10576703399419785, 0...",None,5,OTHER,6a84abdc3e3745519a5232d6d24d4886,None


In [10]:
from langchain_huggingface import HuggingFaceEmbeddings

embedding_handler = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")  
embedding = embedding_handler.embed_query("Hello, world!")

print(embedding)

[0.03492268547415733, 0.018829984590411186, -0.017854733392596245, 0.00013884168583899736, 0.07407364249229431, -0.02262638881802559, 0.003907605540007353, 0.039199575781822205, 0.03311212360858917, -0.014111749827861786, -0.028806891292333603, -0.037598636001348495, -0.03230194374918938, 0.02541874535381794, 0.0883222296833992, -0.07748756557703018, 0.0674402192234993, -0.010266470722854137, -0.040084488689899445, 0.009139186702668667, 0.011325559578835964, 0.007057362701743841, 0.016745109111070633, 0.05523587763309479, 0.01936730556190014, -0.027065211907029152, 0.031041448935866356, -0.0007472320576198399, 0.024931034073233604, 0.005939629394561052, -0.033442385494709015, 0.0035189646296203136, 0.04953842982649803, 0.05694106966257095, 2.0608777049346827e-06, -0.04632243886590004, 0.022610342130064964, -0.013061339035630226, -0.008191742934286594, -0.0029913613107055426, -0.009296656586229801, 0.02691483683884144, -0.019516760483384132, 0.010469090193510056, 0.0029321513138711452, 

In [20]:
fetched_docs['embedding'] = fetched_docs['text'].apply(lambda x: embedding_handler.embed_query(x))

fetched_docs.head()


,id,text,metadata,parent_document_id,type,embedding,keywords,ref_in_parent,tag,previous_document_id,next_document_id
0,7949783dcb314b838fcb409011d7165e,"![](data:image/png;base64,iVBORw0KGgoAAAANSUhE...","{'headings': [], 'images': False, 'tables': Fa...",42f77327d9f14b50848025b4bc8bd4df,PAGE,"[0.024130431935191154, 0.005421559326350689, -...",None,1,OTHER,None,None
1,5ea1b169b5034e3c97de706e4d1e5785,"\n\n![](data:image/png;base64,iVBORw0KGgoAAAAN...","{'headings': [], 'images': False, 'tables': Fa...",42f77327d9f14b50848025b4bc8bd4df,PAGE,"[0.020235136151313782, 0.008043157868087292, -...",None,2,OTHER,7949783dcb314b838fcb409011d7165e,None
2,76e27080698e46d0a6ee83b18bb2254e,\n\nIn our constant endeavour to provide assis...,"{'headings': [], 'images': False, 'tables': Fa...",42f77327d9f14b50848025b4bc8bd4df,PAGE,"[0.0057883476838469505, -0.046952418982982635,...",None,3,OTHER,5ea1b169b5034e3c97de706e4d1e5785,None
3,6a84abdc3e3745519a5232d6d24d4886,"\n\n### Dear Customer,\n\nWelcome to the TATA ...","{'headings': [], 'images': False, 'tables': Fa...",42f77327d9f14b50848025b4bc8bd4df,PAGE,"[0.015528478659689426, -0.07098111510276794, 0...",None,4,OTHER,76e27080698e46d0a6ee83b18bb2254e,None
4,ad92cc51e8614359ae23c3b9c0caccbe,"\n\n- Before driving, read this Owner’s manual...","{'headings': [], 'images': False, 'tables': Fa...",42f77327d9f14b50848025b4bc8bd4df,PAGE,"[0.025854503735899925, -0.10576703399419785, 0...",None,5,OTHER,6a84abdc3e3745519a5232d6d24d4886,None


In [34]:
# Drop existing table and recreate with updated embeddings
if "cars_docs_data" in db.table_names():
    db.drop_table("cars_docs_data")

# Create new table with the updated dataframe that includes embeddings
db.create_table("cars_docs_data", data=fetched_docs.to_dict('records'))


LanceTable(connection=LanceDBConnection(c:\Users\chyav\Documents\Sakaar\lumona_car\notebooks\data\sample-lancedb), name="cars_docs_data")

In [35]:
import pyarrow as pa

# Convert embeddings to fixed-size list type
embedding_length = len(fetched_docs['embedding'].iloc[0])
embedding_type = pa.list_(pa.float32(), embedding_length)

# Create table schema with vector type for embeddings
schema = pa.schema([
    pa.field('id', pa.string()),
    pa.field('text', pa.string()),
    pa.field('parent_document_id', pa.string()),
    pa.field('type', pa.string()),
    pa.field('embedding', embedding_type),
    pa.field('keywords', pa.string()),
    pa.field('ref_in_parent', pa.int64()),
    pa.field('tag', pa.string()),
    pa.field('previous_document_id', pa.string()),
    pa.field('next_document_id', pa.string())
])

# Convert DataFrame to Arrow Table with schema
arrow_table = pa.Table.from_pandas(fetched_docs, schema=schema)

# Drop existing table and recreate with proper schema
if "cars_docs_data" in db.table_names():
    db.drop_table("cars_docs_data")

# Create new table with vector type for embeddings
db.create_table("cars_docs_data", data=arrow_table)

# Open table for later use
rag_table = db.open_table("cars_docs_data")


In [26]:
question = "What is the maximum speed of the car?"

question_embedding = embedding_handler.embed_query(question)

print(question_embedding)

[0.009674747474491596, -0.06683123111724854, 0.033189352601766586, 0.0554855614900589, 0.0009304180857725441, 0.003025803482159972, -0.03186558932065964, 0.08672241121530533, -0.012017900124192238, -0.014724534004926682, 0.021489053964614868, 0.037015195935964584, -0.020237216725945473, 0.02765250764787197, -0.022318486124277115, 0.0025829067453742027, 0.035245630890131, -0.013251326978206635, -0.013965275138616562, -0.00857576448470354, -0.014753867872059345, -0.010502503253519535, 0.01350544672459364, -0.01898815855383873, -0.02397085539996624, 0.012047377415001392, 0.06656132638454437, -0.026562290266156197, 0.015674199908971786, -0.0017597689293324947, -0.013696655631065369, 0.014863156713545322, 0.07202032208442688, 0.08626570552587509, 1.4676264754598378e-06, -0.01976739801466465, 0.031804196536540985, -0.009886966086924076, 0.007860418409109116, 0.018785081803798676, 0.024870119988918304, 0.03978055343031883, -0.033984068781137466, -0.042927082628011703, -0.028721481561660767, -

In [36]:
# Convert list to numpy array for vector search
import numpy as np
query_vector = np.array(question_embedding, dtype=np.float32)

# Build query schema based on the question
# Build query conditions based on non-None values
query = {
    "keywords": ["speed", "maximum"],
    "car_model": None,  # Could be extracted from question if needed
    "tags": ["TECHNICAL"],  # Assuming technical specs would contain speed info
    "type": "TABLE"  # Speed likely found in technical tables
}

# Build WHERE clause dynamically based on non-None values
where_conditions = []
if query["type"]:
    where_conditions.append(f"type = '{query['type']}'")
if query["tags"]:
    where_conditions.append(f"tag IN {query['tags']}")
if query["car_model"]:
    where_conditions.append(f"car_model = '{query['car_model']}'")

# Join conditions with AND
where_clause = " AND ".join(where_conditions)

# Combine vector search with metadata filtering
retrieved_docs = (
    rag_table.search(query_vector, vector_column_name="embedding")
    .where(where_clause)
    .limit(10)
    .to_pandas()
)

retrieved_docs.head()

,id,text,parent_document_id,type,embedding,keywords,ref_in_parent,tag,previous_document_id,next_document_id,_distance
0,1da03904656c461c99e0b8772c6296e2,\n\n|TECHNICAL INFORMATION|Col2|Col3|\n|---|--...,42f77327d9f14b50848025b4bc8bd4df,PAGE,"[-0.00024262775, -0.077529095, 0.023422148, 0....",None,131,OTHER,30846e3e5b6a4be0a5bef1ce909310ab,None,0.884154
1,c7871c101f034527a2a3ce4103b637e6,|TECHNICAL INFORMATION|Col2|Col3|\n|---|---|--...,1da03904656c461c99e0b8772c6296e2,TABLE,"[-0.00024262775, -0.077529095, 0.023422148, 0....",None,131,OTHER,None,None,0.884154
2,f398c58d1a1341ccbfaa855a81ac2160,\n\n### Technical Specifications\n\n|Technical...,42f77327d9f14b50848025b4bc8bd4df,PAGE,"[-0.008696416, -0.05095977, 0.021224033, 0.072...",None,129,OTHER,a6f3bd8594ec444abe2026a4ec7fab66,None,1.062284
3,c96d35f05022459197a239915d27b7ba,|Gear|Diesel|Petrol|\n|---|---|---|\n||Speed(k...,474e0aba5b634d0ea39deb5024623cb6,TABLE,"[-0.040099915, -0.011716243, -0.0056058257, 0....",None,71,OTHER,None,None,1.084443
4,ceabade9777e44cbbc1c3988e988f7dc,|Technical Specifications|Col2|Col3|\n|---|---...,f398c58d1a1341ccbfaa855a81ac2160,TABLE,"[-0.009148891, -0.048287578, 0.014036992, 0.07...",None,129,OTHER,None,None,1.101092
